In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install -q ktrain

     |████████████████████████████████| 421.8MB 39kB/s 
     |████████████████████████████████| 6.7MB 41.8MB/s 
     |████████████████████████████████| 983kB 33.7MB/s 
     |████████████████████████████████| 245kB 42.1MB/s 
     |████████████████████████████████| 675kB 38.4MB/s 
     |████████████████████████████████| 471kB 37.4MB/s 
     |████████████████████████████████| 450kB 36.1MB/s 
     |████████████████████████████████| 3.9MB 40.0MB/s 
     |████████████████████████████████| 890kB 35.2MB/s 
     |████████████████████████████████| 1.1MB 23.8MB/s 
     |████████████████████████████████| 3.8MB 37.6MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
train_b=pd.read_csv("/content/drive/My Drive/minor/hindi_dataset/hindi_dataset.tsv",sep='\t',encoding='utf-8') #training data
train_b=train_b[train_b.task_3!='NONE']
test_b=pd.read_csv("/content/drive/My Drive/minor/hindi_dataset/hasoc2019_hi_test_gold_2919.tsv",sep='\t',encoding='utf-8') #testing data
test_b=test_b[test_b.task_3!='NONE']
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_3'],test_size=0.15,random_state=42) #training and validation split
target_names=['TIN','UNT'] #names of classes

In [5]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [6]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-multilingual-cased' #bert predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for bert
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get bert classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for bert

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: hi
train sequence lengths:
	mean : 28
	95percentile : 57
	99percentile : 62


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:520: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: hi
test sequence lengths:
	mean : 29
	95percentile : 57
	99percentile : 61


In [7]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 350 steps, validate for 12 steps
Epoch 1/4
350/350 [==============================] - 551s 2s/step - loss: 0.6710 - accuracy: 0.6125 - val_loss: 0.6576 - val_accuracy: 0.6550
Epoch 2/4
350/350 [==============================] - 531s 2s/step - loss: 0.6737 - accuracy: 0.6187 - val_loss: 0.6761 - val_accuracy: 0.6550
Epoch 3/4
350/350 [==============================] - 531s 2s/step - loss: 0.6717 - accuracy: 0.6153 - val_loss: 0.6527 - val_accuracy: 0.6550
Epoch 4/4
350/350 [==============================] - 531s 2s/step - loss: 0.6667 - accuracy: 0.6206 - val_loss: 0.6523 - val_accuracy: 0.6550


In [8]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       243
           1       0.00      0.00      0.00       128

    accuracy                           0.65       371
   macro avg       0.33      0.50      0.40       371
weighted avg       0.43      0.65      0.52       371



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([[243,   0],
       [128,   0]])

In [9]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  bert

In [10]:
predictor.get_classes()

['TIN', 'UNT']

In [17]:
test_b_text=list(test_b['text']) #get text for testing
test_b['task_3']=test_b['task_3'].map({'UNT':1,'TIN':0})
test_b_label=list(test_b['task_3']) #get label of testing
# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [18]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [19]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [20]:
from sklearn.metrics import classification_report

In [21]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       542
           1       0.00      0.00      0.00        63

    accuracy                           0.90       605
   macro avg       0.45      0.50      0.47       605
weighted avg       0.80      0.90      0.85       605



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
